In [11]:
import pandas as pd
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# --- CONFIG ---
KEYWORD = "Cafe Banyuwangi"
SCROLL_COUNT = 10  # Dinaikin biar lebih banyak data

chrome_options = Options()
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
# Optimasi speed
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def smart_sleep(min_sec, max_sec):
    time.sleep(random.uniform(min_sec, max_sec))

print(f"🚀 Memulai Scraping: {KEYWORD}")

try:
    driver.get(f"https://www.google.com/maps/search/{KEYWORD}")
    smart_sleep(5, 7)

    # 1. Proses Scrolling dengan cara lebih aman
    print("📜 Scrolling untuk load semua data...")
    for i in range(SCROLL_COUNT):
        try:
            # Cari panel dengan berbagai cara
            panel = None
            try:
                panel = driver.find_element(By.XPATH, '//div[@role="feed"]')
            except:
                try:
                    panel = driver.find_element(By.CSS_SELECTOR, 'div[role="feed"]')
                except:
                    print("   ⚠️  Panel tidak ditemukan, skip scroll")
                    break
            
            # Scroll ke bawah
            driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', panel)
            smart_sleep(2, 4)
            print(f"   [{i+1}/{SCROLL_COUNT}] Scrolling...")
            smart_sleep(4, 7)
            
            # Cek kalau udah mentok (ada tulisan "You've reached the end")
            try:
                driver.find_element(By.XPATH, '//*[contains(text(), "reached the end")]')
                print("   ✓ Sudah sampai akhir daftar")
                break
            except:
                pass
                
        except Exception as e:
            print(f"   Error saat scroll: {e}")
            break

    # 2. Ekstraksi Data dengan Multiple Strategy
    print("\n📦 Mengekstrak data...")
    all_results = []
    
    # Strategy 1: Ambil semua link (paling reliable)
    try:
        # Tunggu sampai elemen muncul
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href*='maps/place']"))
        )
        
        # Ambil semua link bisnis
        links = driver.find_elements(By.CSS_SELECTOR, "a[href*='maps/place']")
        print(f"🔍 Ditemukan {len(links)} potential entries")
        
        seen_urls = set()
        
        for idx, link in enumerate(links):
            try:
                url = link.get_attribute("href")
                
                # Skip duplikat
                if url in seen_urls:
                    continue
                seen_urls.add(url)
                
                # Ambil aria-label untuk nama (ini paling reliable)
                nama = link.get_attribute("aria-label")
                if not nama or len(nama) < 2:
                    continue
                
                print(f"\n[{idx+1}] Processing: {nama}")
                
                # Cari parent container untuk ambil alamat
                alamat = "Tidak ditemukan"
                rating = "N/A"
                
                try:
                    # Naik ke parent container
                    parent = link.find_element(By.XPATH, "./ancestor::div[contains(@class, 'Nv2PK') or contains(@class, 'THOPZb')]")
                    
                    # Cari semua span di dalam parent
                    all_spans = parent.find_elements(By.TAG_NAME, "span")
                    
                    for span in all_spans:
                        text = span.text.strip()
                        if not text:
                            continue
                        
                        # Filter alamat: cari yang ada keyword lokasi dan cukup panjang
                        if len(text) > 15:  # Alamat biasanya panjang
                            if any(keyword in text.lower() for keyword in 
                                   ["jl.", "jalan", "kec.", "kab.", "banyuwangi", "no.", "gg.", "gang"]):
                                alamat = text
                                print(f"   📍 Alamat: {alamat}")
                        
                        # Filter rating: cari pola angka dengan koma/titik
                        if len(text) <= 5 and (',' in text or '.' in text):
                            try:
                                # Cek apakah bisa diconvert ke float
                                float(text.replace(',', '.'))
                                rating = text
                                print(f"   ⭐ Rating: {rating}")
                            except:
                                pass
                
                except Exception as e:
                    print(f"   ⚠️  Gagal ambil detail: {e}")
                
                # Simpan data
                all_results.append({
                    "nama_bisnis": nama,
                    "rating": rating,
                    "alamat": alamat,
                    "link": url
                })
                print(f"   ✅ Data tersimpan")
                
            except Exception as e:
                print(f"   ❌ Error pada entry: {e}")
                continue
    
    except Exception as e:
        print(f"❌ Error saat ekstraksi: {e}")

    # 3. Simpan ke Excel
    if len(all_results) > 0:
        df = pd.DataFrame(all_results)
        
        # Bersihkan duplikat berdasarkan nama dan alamat
        df.drop_duplicates(subset=['nama_bisnis', 'alamat'], inplace=True)
        
        # Sort berdasarkan rating (yang ada rating di atas)
        df['rating_sort'] = df['rating'].apply(lambda x: 0 if x == 'N/A' else float(x.replace(',', '.')))
        df = df.sort_values('rating_sort', ascending=False)
        df = df.drop('rating_sort', axis=1)
        
        # Simpan
        filename = "database_cafe_bwi_fixed2.xlsx"
        df.to_excel(filename, index=False)

        print("\n" + "="*50)
        print(f"✨ BERHASIL! Dapet {len(df)} data unik.")
        print(f"📂 File disimpan: {filename}")
        print("="*50)
        
        # Preview data
        print("\n📊 Preview 5 data pertama:")
        print(df.head().to_string())
    else:
        print("\n❌ GAGAL! Tidak ada data yang berhasil di-scrape.")
        print("💡 Tips:")
        print("   - Coba jalankan ulang (Google Maps suka load lambat)")
        print("   - Pastikan koneksi internet stabil")
        print("   - Chrome driver sudah terupdate")

except Exception as e:
    print(f"\n❌ ERROR FATAL: {e}")
    import traceback
    traceback.print_exc()

finally:
    print("\n🔚 Menutup browser...")
    driver.quit()
    print("Done!")

🚀 Memulai Scraping: Cafe Banyuwangi
📜 Scrolling untuk load semua data...
   [1/10] Scrolling...
   [2/10] Scrolling...
   [3/10] Scrolling...
   [4/10] Scrolling...
   [5/10] Scrolling...
   [6/10] Scrolling...
   [7/10] Scrolling...
   [8/10] Scrolling...
   [9/10] Scrolling...
   [10/10] Scrolling...

📦 Mengekstrak data...
🔍 Ditemukan 58 potential entries

[1] Processing: Erenka Cafe & Space
   ⭐ Rating: 4,6
   📍 Alamat: · Jl. Jember No.64
   📍 Alamat: Jl. Jember No.64
   ✅ Data tersimpan

[2] Processing: Sakaloka Cafe
   ⭐ Rating: 4,7
   ✅ Data tersimpan

[3] Processing: 1911 Cafe & Resto
   ⭐ Rating: 4,6
   📍 Alamat: · PT, Jl. K Muhammad Jl. Perkebunan Kalibendo No.Desa
   📍 Alamat: PT, Jl. K Muhammad Jl. Perkebunan Kalibendo No.Desa
   ✅ Data tersimpan

[4] Processing: Atap Langit Coffee
   ⭐ Rating: 4,8
   ✅ Data tersimpan

[5] Processing: Ujung Jawa Cafe & Eatery
   ⭐ Rating: 4,7
   📍 Alamat: · Jl. Mh Thamrin No.40
   📍 Alamat: Jl. Mh Thamrin No.40
   ✅ Data tersimpan

[6] Proce